In [17]:
# ALPLA demo- Implementation with a small scale dataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import OneClassSVM

# import sys
# print("Python version")
# print(sys.version)


np.set_printoptions(threshold=np.inf)

# Load measurement file:
measurement = np.load('dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']

# print(header.dtype)
print(data['cirs'].shape)


(9797, 15, 251, 2)


In [18]:

# Understanding Array
# Our testing CIR dataset
new_data = {
    'cirs': np.array(
        [
            [
                [
                    [1, 2],
                    [3, 4],
                    [5, 6]
                ],
                [
                    [9, 10],
                    [11, 12],
                    [13, 14]
                ],
                [
                    [17, 18],
                    [19, 20],
                    [21, 22]
                ]
            ],
            [
                [
                    [25, 26],
                    [27, 28],
                    [29, 30]
                ],
                [
                    [33, 34],
                    [35, 36],
                    [37, 38]
                ],
                [
                    [41, 42],
                    [43, 44],
                    [45, 46]
                ]
            ],
            [
                [
                    [49, 50],
                    [51, 52],
                    [53, 54]
                ],
                [
                    [57, 58],
                    [59, 60],
                    [61, 62]
                ],
                [
                    [65, 66],
                    [67, 68],
                    [69, 70]
                ]
            ]
        ]
    )
}


In [19]:
# Step 1.1 - Extract real and imaginary parts for Alice and Eve (assuming channels 0 and 1)
alice_real = new_data['cirs'][:, 0, :, 0]
alice_imag = new_data['cirs'][:, 0, :, 1]

eve_real = new_data['cirs'][:, 1, :, 0]
eve_imag = new_data['cirs'][:, 1, :, 1]

In [20]:
# Print
print('Alice Real:')
print(alice_real)
print('Alice Imaginary:')
print(alice_imag)

print()

print('Eve Real:')
print(eve_real)
print('Eve Imaginary:')
print(eve_imag)

Alice Real:
[[ 1  3  5]
 [25 27 29]
 [49 51 53]]
Alice Imaginary:
[[ 2  4  6]
 [26 28 30]
 [50 52 54]]

Eve Real:
[[ 9 11 13]
 [33 35 37]
 [57 59 61]]
Eve Imaginary:
[[10 12 14]
 [34 36 38]
 [58 60 62]]


In [21]:
# Step 1.2 - Compute magnitudes
alice_mag = np.sqrt(alice_real**2 + alice_imag**2)
eve_mag = np.sqrt(eve_real**2 + eve_imag**2)

In [22]:
# Print
print('Alice Magnitude:')
print(alice_mag)

print()

print('Eve Magnitude:')
print(eve_mag)

Alice Magnitude:
[[ 2.23606798  5.          7.81024968]
 [36.06937759 38.89730068 41.72529209]
 [70.00714249 72.83543094 75.66372975]]

Eve Magnitude:
[[13.45362405 16.2788206  19.10497317]
 [47.38143096 50.20956084 53.03772242]
 [81.32035416 84.14867795 86.97700846]]


In [23]:
# Step 1.3 - Flatten the arrays

# Flatten features for Alice
alice_real_flat = alice_real.flatten()
alice_imag_flat = alice_imag.flatten()
alice_mag_flat = alice_mag.flatten()

# Flatten features for Eve
eve_real_flat = eve_real.flatten()
eve_imag_flat = eve_imag.flatten()
eve_mag_flat = eve_mag.flatten()

# We flatten the arrays to make them suitable for the MinMaxScaler
# We flat all three(real, imaginary and magnitude) parts of the signal

In [24]:
# Print
print('Alice Real Flat:')
print(alice_real_flat)

print()

print('Alice Imaginary Flat:')
print(alice_imag_flat)

print()

print('Alice Magnitude Flat:')
print(alice_mag_flat)

print() 
print()

print('Eve Real Flat:')
print(eve_real_flat)

print()

print('Eve Imaginary Flat:')
print(eve_imag_flat)

print()

print('Eve Magnitude Flat:')
print(eve_mag_flat)


Alice Real Flat:
[ 1  3  5 25 27 29 49 51 53]

Alice Imaginary Flat:
[ 2  4  6 26 28 30 50 52 54]

Alice Magnitude Flat:
[ 2.23606798  5.          7.81024968 36.06937759 38.89730068 41.72529209
 70.00714249 72.83543094 75.66372975]


Eve Real Flat:
[ 9 11 13 33 35 37 57 59 61]

Eve Imaginary Flat:
[10 12 14 34 36 38 58 60 62]

Eve Magnitude Flat:
[13.45362405 16.2788206  19.10497317 47.38143096 50.20956084 53.03772242
 81.32035416 84.14867795 86.97700846]


In [25]:
# Step 1.4 - Combine features (making a feature vector)
alice_features = np.column_stack((alice_real_flat, alice_imag_flat, alice_mag_flat))
eve_features = np.column_stack((eve_real_flat, eve_imag_flat, eve_mag_flat))

# By doing np.column_stack, we are making a feature vector of the signal (combining all three parts of the signal)

In [26]:
print('Alice Combined Features:')
print(alice_features)

print()

print('Eve Combined Features:')
print(eve_features)

Alice Combined Features:
[[ 1.          2.          2.23606798]
 [ 3.          4.          5.        ]
 [ 5.          6.          7.81024968]
 [25.         26.         36.06937759]
 [27.         28.         38.89730068]
 [29.         30.         41.72529209]
 [49.         50.         70.00714249]
 [51.         52.         72.83543094]
 [53.         54.         75.66372975]]

Eve Combined Features:
[[ 9.         10.         13.45362405]
 [11.         12.         16.2788206 ]
 [13.         14.         19.10497317]
 [33.         34.         47.38143096]
 [35.         36.         50.20956084]
 [37.         38.         53.03772242]
 [57.         58.         81.32035416]
 [59.         60.         84.14867795]
 [61.         62.         86.97700846]]


In [27]:
# Step 1.5 - Normalize the data (to train the model)
scaler = MinMaxScaler()
alice_features_scaled = scaler.fit_transform(alice_features)
eve_features_scaled = scaler.fit_transform(eve_features) # Used the samne way as Alice's data was scaled

In [28]:
# Print
print('Alice Scaled Features:')
print(alice_features_scaled)

print()

print('Eve Scaled Features:')
print(eve_features_scaled)

Alice Scaled Features:
[[0.         0.         0.        ]
 [0.03846154 0.03846154 0.03764156]
 [0.07692308 0.07692308 0.07591392]
 [0.46153846 0.46153846 0.46077063]
 [0.5        0.5        0.49928367]
 [0.53846154 0.53846154 0.53779765]
 [0.92307692 0.92307692 0.92296381]
 [0.96153846 0.96153846 0.96148183]
 [1.         1.         1.        ]]

Eve Scaled Features:
[[0.         0.         0.        ]
 [0.03846154 0.03846154 0.03842582]
 [0.07692308 0.07692308 0.07686465]
 [0.46153846 0.46153846 0.461456  ]
 [0.5        0.5        0.49992172]
 [0.53846154 0.53846154 0.53838787]
 [0.92307692 0.92307692 0.92306319]
 [0.96153846 0.96153846 0.96153155]
 [1.         1.         1.        ]]


In [29]:
# Step 2.1 - Train OCC-SVM for Alice
occ_svm = OneClassSVM(kernel='linear', nu=0.01)  # You can experiment with other kernels like 'rbf', 'poly', 'sigmoid'
occ_svm.fit(alice_features_scaled)

import joblib
joblib.dump(occ_svm, 'occ_svm_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [30]:
# Step 3.1 - Test Incoming Signals
occ_svm = joblib.load('occ_svm_model.pkl')
scaler = joblib.load('scaler.pkl')

# Example incoming signal data for testing (using the same dataset here for simplicity)
incoming_data = new_data['cirs']

# Extract features for testing
incoming_real = incoming_data[:, 0, :, 0]
incoming_imag = incoming_data[:, 0, :, 1]
incoming_mag = np.sqrt(incoming_real**2 + incoming_imag**2)

# Combine and normalize features
incoming_features_vector = np.column_stack((incoming_real.flatten(), incoming_imag.flatten(), incoming_mag.flatten()))
# Scaled the incoming features using the same scaler used for Alice
incoming_features_scaled = scaler.transform(incoming_features_vector)

In [31]:
# Predict using OCC-SVM
predictions = occ_svm.predict(incoming_features_scaled)

In [32]:
# Check predictions (1 for inliers/legitimate, -1 for outliers/illegitimate)
print(predictions)
legitimate = predictions == 1
illegitimate = predictions == -1
print("Legitimate signals:", legitimate.sum())
print("Illegitimate signals:", illegitimate.sum())

[-1 -1 -1 -1 -1 -1 -1 -1 -1]
Legitimate signals: 0
Illegitimate signals: 9
